In [1]:
!pip install kdbai_client --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.


In [2]:
!git clone https://github.com/facebookresearch/ImageBind

Cloning into 'ImageBind'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 146 (delta 62), reused 50 (delta 40), pack-reused 47 (from 1)
Receiving objects: 100% (146/146), 2.64 MiB | 6.04 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [3]:
import os
os.chdir('./ImageBind')

In [4]:
!pip install .

Processing /content/ImageBind
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-install-xdekqpbl/pytorchvideo_d74b4dc23b9143ccb7d99c3fa82250e4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-xdekqpbl/pytorchvideo_d74b4dc23b9143ccb7d99c3fa82250e4
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.p

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os
import PIL
from PIL import Image
import torch
from imagebind import data
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)
print("")

  0%|          | 0.00/4.47G [00:00<?, ?B/s]

In [8]:
import csv
def read_context(filename):
  context = {}
  with open(filename, 'r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    for row in csv_reader:
      context[row[0]] = row[1]
  return context

context = read_context('/content/drive/MyDrive/LLM_Mid1/context.csv')

In [10]:
def getEmbeddingVector(inputs):
    with torch.no_grad():
        embedding = model(inputs)
    for key, value in embedding.items():
        vec = value.reshape(-1)
        vec = vec.cpu().numpy()
        return(vec)

def dataToEmbedding(dataIn,dtype):
    if dtype == 'image':
        data_path = [dataIn]
        inputs = {
        ModalityType.VISION: data.load_and_transform_vision_data(data_path, device)
        }
    elif dtype == 'text':
        txt = [dataIn]
        inputs = {
        ModalityType.TEXT: data.load_and_transform_text(txt, device)
        }
    vec = getEmbeddingVector(inputs)
    return(vec)

def queryToEmbedding(dataIn):
    data_path = [dataIn]
    inputs = {
        ModalityType.VISION: data.load_and_transform_vision_data(data_path, device)
    }
    vec = getEmbeddingVector(inputs)
    return(vec)

In [11]:
columns = ['path', 'embeddings', 'context']
df = pd.DataFrame(columns=columns)

In [12]:
for image in os.listdir('/content/drive/MyDrive/LLM_Mid1/Images'):
    path = f'/content/drive/MyDrive/LLM_Mid1/Images/{image}'
    name = image.split('.')[0]
    con = context[name]
    embedding = dataToEmbedding(path, 'image')
    new_row = {'path': path,
               'embeddings':embedding,
               'context':con}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [14]:
import os
from getpass import getpass
import kdbai_client as kdbai
import time

In [15]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

KDB.AI endpoint: https://cloud.kdb.ai/instance/ad32inc8wq
KDB.AI API key: ··········


In [16]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

In [17]:
table_schema = {
    "columns": [
        {"name": "path", "pytype": "str"},
        {
            "name": "embeddings",
            "pytype": "float64",
            "vectorIndex": {"dims": 1024, "metric": "CS", "type": "flat"},
        },
        {"name": "context", "pytype": "str"},
    ]
}

In [18]:
try:
    session.table("LLM").drop()
    time.sleep(5)
except kdbai.KDBAIException:
    pass

In [19]:
table = session.create_table("LLM", table_schema)

In [20]:
from tqdm import tqdm
n = 2000
for i in tqdm(range(0, df.shape[0], n)):
    table.insert(df[i:i+n].reset_index(drop=True))

100%|██████████| 1/1 [00:00<00:00,  2.41it/s]


### Query

In [28]:
session2 = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

In [29]:
table = session2.table("LLM")

In [30]:
def mm_search(query):
    results = table.search(query, n=1)
    return(results)

In [31]:
query_vector = [queryToEmbedding("/content/Query.jpeg").tolist()]
results = table.search(query_vector, n=1)

In [32]:
results[0].values[0]

array([0.7035256624221802,
       '/content/drive/MyDrive/LLM_Mid1/Images/10.jpeg',
       array([-0.02287591,  0.01611555,  0.02260055, ..., -0.02161041,
              -0.01728459,  0.03062696], dtype=float32)               ,
       ' The image shows the General store of IIIT Delhi'], dtype=object)